In [1]:
import time
import pandas as pd
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

In [2]:
#Students of ENEM
E = pd.read_csv('~/data/enem/2016/DADOS/microdados_enem_2016.csv', sep=';', encoding="iso-8859-2")
E.drop(E.filter(regex='RESPOSTAS|GABARITO').columns, axis=1, inplace = True)

In [3]:
#School census data
CE =pd.read_csv("~/data/censo/2016/DADOS/ESCOLAS.CSV", sep='|', encoding="iso-8859-2")

In [4]:
#Teacher census data by Braazilian mesoregion
cols = ['CO_PESSOA_FISICA','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO','CO_ENTIDADE']
CD_CO = pd.read_csv("~/data/censo/2016/DADOS/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = cols)
CD_NE = pd.read_csv("~/data/censo/2016/DADOS/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = cols)
CD_N =  pd.read_csv("~/data/censo/2016/DADOS/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = cols)
CD_SE = pd.read_csv("~/data/censo/2016/DADOS/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = cols)
CD_S =  pd.read_csv("~/data/censo/2016/DADOS/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = cols)

#All teacher census data
CT = pd.concat([CD_CO, CD_NE, CD_N, CD_SE, CD_S])

In [5]:
#print(E.info(memory_usage='deep')) 
#print(CT.info(memory_usage='deep')) 
#print(CE.info(memory_usage='deep')) 
#

In [6]:
#dowcasting numeric feature of some data to improve memory usage.
obj_cols = E.filter(regex = 'Q\d*', axis=1).columns.to_list()
E[obj_cols] = E[obj_cols].astype('category')

num_cols = E.select_dtypes(include =['int']).columns
E[num_cols]= E[num_cols].apply(pd.to_numeric,downcast='unsigned')


In [9]:
flo_cols = E.select_dtypes(include =['float']).columns
E[flo_cols]= E[flo_cols].apply(pd.to_numeric,downcast='float')

#CT main problems
#num_cols = CT.select_dtypes(include =['int']).columns
#CT[num_cols]= CT[num_cols].apply(pd.to_numeric,downcast='unsigned')

#flo_cols = CT.select_dtypes(include =['float']).columns
#CT[flo_cols]= CT[flo_cols].apply(pd.to_numeric,downcast='float')

In [10]:
#obj_cols = CT.select_dtypes(include =['object']).columns
#CT[obj_cols] = CT[obj_cols].astype('category')

obj_cols = CE.select_dtypes(include =['object']).columns
CE[obj_cols] = CE[obj_cols].astype('category')

#ID columns = already known as categorical
id_cols = CE.filter(regex = 'ID', axis=1).columns.to_list()
CE[id_cols] = CE[id_cols].astype('category')

In [13]:
#removing faulty columns to convert the frame to fast type. This columns won't be useful.
CE.drop('CO_ORGAO_REGIONAL', inplace = True, axis= 1)

In [14]:
#Write on disk the typed and faster data.
E.to_feather('E.feather')
CT.reset_index().to_feather ('CT.feather')
CE.to_feather ('CE.feather')


### If it is not the first time, start from this point

In [70]:
import time
import pandas as pd
import numpy as np
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)


ENEM = pd.read_feather('E.feather')
C_TEACHER = pd.read_feather('CT.feather')
C_SCHOOL  = pd.read_feather ('CE.feather')

There are many insignificant features to our work, like codes, PK, FK and out of context. 
Based on prior works and experience of especialists, will be taken off nonsense features. 
A study was carried out on the evolution and correspondence of these resources in the ENEM and Census
data for the period 2009-2018. See (featuresOrganisaton.xls)

In [71]:
teacher = ['CO_PESSOA_FISICA', 'CO_ENTIDADE','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO']

C_TEACHER = C_TEACHER[teacher]

# New default column names for all Datasets
newNames = [
'CO_PROFESSOR',
'CO_ESCOLA',  
'IN_ESPECIALIZACAO',
'IN_MESTRADO',
'IN_DOUTORADO'
]
#Rename
C_TEACHER.columns = newNames
#convert Keys to Str
C_TEACHER.CO_PROFESSOR = C_TEACHER.CO_PROFESSOR.astype('int32')
C_TEACHER.CO_ESCOLA = C_TEACHER.CO_ESCOLA.astype('int32')

In [72]:
school = ['NU_ANO_CENSO','CO_ENTIDADE', 'CO_MUNICIPIO','CO_UF','TP_DEPENDENCIA','IN_AGUA_INEXISTENTE',
                  'IN_ESGOTO_INEXISTENTE','IN_ENERGIA_INEXISTENTE',
                   'IN_LABORATORIO_INFORMATICA','IN_LABORATORIO_CIENCIAS','IN_SALA_ATENDIMENTO_ESPECIAL',
                   'IN_BIBLIOTECA','IN_SALA_LEITURA','IN_BANHEIRO_FORA_PREDIO',
                   'IN_BANHEIRO_PNE','IN_DEPENDENCIAS_PNE','NU_SALAS_UTILIZADAS','NU_EQUIP_TV',
                   'NU_EQUIP_DVD','NU_EQUIP_COPIADORA','NU_EQUIP_IMPRESSORA','NU_COMP_ADMINISTRATIVO',
                   'NU_COMP_ALUNO','IN_INTERNET','IN_BANDA_LARGA','NU_FUNCIONARIOS','IN_ALIMENTACAO']

C_SCHOOL = C_SCHOOL[school]


# New default column names for all Datasets
newNames = ['ANO_CENSO',
'CO_ESCOLA',
 'CO_MUNICIPIO',
 'CO_UF',
 'CO_DEPENDENCIA_ADM',
 'IN_AGUA_INEXISTENTE',
 'IN_ESGOTO_INEXISTENTE',
 'IN_ENERGIA_INEXISTENTE',
 'IN_LABORATORIO_INFORMATICA',
 'IN_LABORATORIO_CIENCIAS',
 'IN_SALA_ATENDIMENTO_ESPECIAL',
 'IN_BIBLIOTECA',
 'IN_SALA_LEITURA',
 'IN_BANHEIRO_FORA_PREDIO',
 'IN_BANHEIRO_PNE',
 'IN_DEPENDENCIAS_PNE',
 'QT_SALAS_UTILIZADAS',
 'QT_EQUIP_TV',
 'QT_EQUIP_DVD',
 'QT_EQUIP_COPIADORA',
 'QT_EQUIP_IMPRESSORA',
 'QT_COM_ADMINISTRATIVO',
 'QT_COMP_ALUNO',
 'IN_INTERNET',
 'IN_BANDA_LARGA',
 'QT_FUNCIONARIOS',
 'IN_ALIMENTACAO'
]

#Rename 
C_SCHOOL.columns =  newNames
#convert Keys to Str
C_SCHOOL.CO_ESCOLA = C_SCHOOL.CO_ESCOLA.astype('int32')
print('Checking year of censo',C_SCHOOL.ANO_CENSO.unique())
C_SCHOOL.drop('ANO_CENSO', axis=1, inplace=True)

Checking year of censo [2016]


In [73]:
enem = ['NU_ANO','CO_ESCOLA','TP_ENSINO','NU_IDADE','TP_SEXO','TP_COR_RACA','TP_ST_CONCLUSAO','NU_NOTA_CN',
        'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','Q001','Q002','Q005','Q006'
]


ENEM = ENEM[enem]

# New default column names for all Datasets
newNames = [
'CO_ANO',
'CO_ESCOLA',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'TP_ST_CONCLUSAO',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL'
]

#Rename
ENEM.columns = newNames

In [74]:
#NO
for i in ENEM.select_dtypes(include =['category']).columns:
    print(i, list(ENEM[i].value_counts().index))

EDU_PAI ['B', 'E', 'C', 'D', 'H', 'A', 'F', 'G']
EDU_MAE ['E', 'B', 'C', 'D', 'F', 'G', 'A', 'H']
QT_PESSOAS_CASA [4.0, 3.0, 5.0, 2.0, 6.0, 7.0, 1.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 20.0, 15.0, 16.0, 19.0, 18.0, 17.0]
RENDA_MENSAL ['C', 'B', 'D', 'E', 'G', 'F', 'H', 'A', 'I', 'J', 'N', 'K', 'Q', 'O', 'P', 'M', 'L']


In [75]:
for i in C_SCHOOL.select_dtypes(include =['category']).columns:
    print(i, list(C_SCHOOL[i].value_counts().index))

### Minimum Scope Definition

In [76]:
ENEM.dropna(inplace = True, axis=0, subset = ['CO_ESCOLA'])
ENEM.CO_ESCOLA = ENEM.CO_ESCOLA.astype('int32')


In [77]:
#should care about the nulls into scope filter features?
filters = (['TP_ST_CONCLUSAO', 'TP_ENSINO'])
ENEM[filters].isnull().sum()
#NO, IT'S LOWER

TP_ST_CONCLUSAO     0
TP_ENSINO          22
dtype: int64

In [78]:
print('drop out',((ENEM.TP_ENSINO!= 1)|(ENEM.TP_ENSINO!= 3)).sum(), 'who not attend regular and prof approachs schools')
ENEM = ENEM.loc[((ENEM.TP_ENSINO== 1)|(ENEM.TP_ENSINO== 3))]
ENEM.drop('TP_ENSINO', inplace = True, axis =1)

drop out 1881282 who not attend regular and prof approachs schools


In [79]:
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
for i in fields:
    print ('null:', i, ENEM[i].isnull().sum())
    print ('zero:', i, (ENEM[i]== 0).sum())
    print( 'stranger "."', i , (ENEM[i]== '.').sum())
    print( 'stranger "*"', i , (ENEM[i]== '*').sum())
    
    #, "and", ENEM.I==0)


null: NU_NOTA_CN 329911
zero: NU_NOTA_CN 591
stranger "." NU_NOTA_CN 0
stranger "*" NU_NOTA_CN 0
null: NU_NOTA_CH 329911
zero: NU_NOTA_CH 277
stranger "." NU_NOTA_CH 0
stranger "*" NU_NOTA_CH 0
null: NU_NOTA_LC 354144
zero: NU_NOTA_LC 709
stranger "." NU_NOTA_LC 0
stranger "*" NU_NOTA_LC 0
null: NU_NOTA_MT 354144
zero: NU_NOTA_MT 1121
stranger "." NU_NOTA_MT 0
stranger "*" NU_NOTA_MT 0
null: NU_NOTA_REDACAO 354144
zero: NU_NOTA_REDACAO 31080
stranger "." NU_NOTA_REDACAO 0
stranger "*" NU_NOTA_REDACAO 0


In [80]:
ENEM.dropna(inplace = True, axis=0, subset=fields)
ENEM = ENEM.loc[~(ENEM[fields] == 0).any(axis=1)]

In [81]:
ENEM[fields] = ENEM[fields].astype('float32')

In [82]:
ENEM[fields].describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,1.480081e+06,1.480081e+06,1.480081e+06,1.480081e+06,1.480081e+06
mean,4.808624e+02,5.355745e+02,5.217820e+02,4.919406e+02,5.506884e+02
std,7.407105e+01,7.383706e+01,6.802874e+01,1.048920e+02,1.354861e+02
min,3.404000e+02,3.199000e+02,2.902000e+02,3.195000e+02,4.000000e+01
25%,4.229000e+02,4.857000e+02,4.744000e+02,4.127000e+02,4.600000e+02
50%,4.657000e+02,5.382000e+02,5.252000e+02,4.681000e+02,5.400000e+02
75%,5.290000e+02,5.874000e+02,5.699000e+02,5.506000e+02,6.200000e+02
max,8.713000e+02,8.591000e+02,8.026000e+02,9.915000e+02,1.000000e+03


In [83]:
#ONLY STUDENTS AT  17 -19 YEAR OLD. WE WANT TO GET ONLY REGULAR STUDENTS THAT ARE IN LAST YEAR OF SECONDARY SCHOOL. 
#THE MICRODADOS HAVE A FIELD TO CONTROL IT. BUT IT IS NOT CONFIDENCE,DUE A TURN OUT TEAACHERS THAT EXTEND ACADEMIC YEAR
print('null', ENEM.NU_IDADE.isnull().sum())
#firts, fill NA with the mean of student school
ENEM.NU_IDADE = ENEM.groupby(ENEM['CO_ESCOLA'])['NU_IDADE'].apply(lambda x: x.fillna(round(x.mean())))
ENEM.NU_IDADE = ENEM.NU_IDADE.astype('int32')
print('taking off',((ENEM.NU_IDADE <17)|(ENEM.NU_IDADE>19)).sum())
ENEM = ENEM.loc[(ENEM.NU_IDADE >16) &  (ENEM.NU_IDADE<20)]
ENEM.NU_IDADE.value_counts()

null 32
taking off 209748


17    721806
18    418035
19    130492
Name: NU_IDADE, dtype: int64

### Transform features to default categories

In [84]:
ENEM.QT_PESSOAS_CASA.fillna(ENEM.QT_PESSOAS_CASA.mode()[0], inplace=True)
ENEM.QT_PESSOAS_CASA = ENEM.QT_PESSOAS_CASA.astype('int32')

In [85]:
fields = ['EDU_PAI', 'EDU_MAE', 'RENDA_MENSAL', 'TP_SEXO', 'TP_COR_RACA']
ENEM[fields].isnull().sum()

EDU_PAI         0
EDU_MAE         0
RENDA_MENSAL    0
TP_SEXO         0
TP_COR_RACA     0
dtype: int64

In [86]:
#Order relation, is better for numeric categories.
for i in fields:
    codes, uniques = pd.factorize(ENEM[i], sort=True)
    ENEM[i] = codes



In [87]:
ENEM['EDU_PAI_TEMP'] = ENEM['EDU_PAI']
ENEM['EDU_MAE_TEMP'] = ENEM['EDU_MAE']
ENEM['QT_PESSOAS_CASA_TEMP'] = ENEM['QT_PESSOAS_CASA']
ENEM['RENDA_MENSAL_TEMP'] = ENEM['RENDA_MENSAL']

In [88]:
# Paring with default categories

ENEM.loc[ENEM['EDU_PAI']==0, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==1, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==2, 'EDU_PAI_TEMP']=1
ENEM.loc[ENEM['EDU_PAI']==3, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==4, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==5, 'EDU_PAI_TEMP']=4
ENEM.loc[ENEM['EDU_PAI']==6, 'EDU_PAI_TEMP']=5
ENEM.loc[ENEM['EDU_PAI']==7, 'EDU_PAI_TEMP']=0

ENEM.loc[ENEM['EDU_MAE']==0, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==1, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==2, 'EDU_MAE_TEMP']=1
ENEM.loc[ENEM['EDU_MAE']==3, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==4, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==5, 'EDU_MAE_TEMP']=4
ENEM.loc[ENEM['EDU_MAE']==6, 'EDU_MAE_TEMP']=5
ENEM.loc[ENEM['EDU_MAE']==7, 'EDU_MAE_TEMP']=0



ENEM.loc[ENEM['QT_PESSOAS_CASA']==1, 'QT_PESSOAS_CASA_TEMP']=0
ENEM.loc[ENEM['QT_PESSOAS_CASA']==2, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==3, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==4, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==5, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==6, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==7, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']>7, 'QT_PESSOAS_CASA_TEMP']=3

ENEM.loc[ENEM['RENDA_MENSAL']==0, 'RENDA_MENSAL_TEMP']=0
ENEM.loc[ENEM['RENDA_MENSAL']==1, 'RENDA_MENSAL_TEMP']=1
ENEM.loc[ENEM['RENDA_MENSAL']==2, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==3, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[(ENEM['RENDA_MENSAL']>3) & (ENEM['RENDA_MENSAL']<8), 'RENDA_MENSAL_TEMP']=3
ENEM.loc[(ENEM['RENDA_MENSAL']>7) & (ENEM['RENDA_MENSAL']<13), 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']>12, 'RENDA_MENSAL_TEMP']=5


ENEM.loc[ENEM['TP_COR_RACA']==6, 'TP_COR_RACA']=0


In [89]:
#drop temps

ENEM.drop(['EDU_PAI_TEMP', 'EDU_MAE_TEMP', 'QT_PESSOAS_CASA_TEMP','RENDA_MENSAL_TEMP'], axis=1, inplace=True)

In [90]:
print('Total Students at ENEM:', ENEM.shape[0], 'of', ENEM.drop_duplicates('CO_ESCOLA').shape[0], 'schools')
print('Total Schools at Census:', C_SCHOOL.shape[0])
print('Total Teachers at Census:', C_TEACHER.drop_duplicates('CO_PROFESSOR').shape[0], 
      'One teacher can be more than one school')
       
            

#######################################
print('########### Only about Schools at ENEM:')

schoolsEnem = ENEM['CO_ESCOLA'].dropna().unique()
schoolsCenso = C_SCHOOL['CO_ESCOLA'].dropna().unique()
schoolsTeacher = C_TEACHER['CO_ESCOLA'].dropna().unique()
schoolsEnem2 = ENEM['CO_ESCOLA'].dropna()

enem_censo = np.setdiff1d(schoolsEnem ,schoolsCenso)
print('Schools at ENEM out of School Census', sum(np.isin(schoolsEnem, schoolsCenso, invert=True)))

print('Schools at ENEM out of Teacher Census', sum(np.isin(schoolsEnem, schoolsTeacher, invert=True)))
print('Schools at ENEM and School Census out of Teacher Census', sum(np.isin(enem_censo, schoolsTeacher)))

print('Students Wasted*****', sum(np.isin(schoolsEnem2, schoolsCenso, invert=True)))

Total Students at ENEM: 1270333 of 30266 schools
Total Schools at Census: 279358
Total Teachers at Census: 2523192 One teacher can be more than one school
########### Only about Schools at ENEM:
Schools at ENEM out of School Census 16610
Schools at ENEM out of Teacher Census 16875
Schools at ENEM and School Census out of Teacher Census 0
Students Wasted***** 689111


In [91]:
print('number of teacher who work at an school listed at ENEM and do not have study level assigned:',
      C_TEACHER['CO_ESCOLA'].isin(ENEM['CO_ESCOLA']).sum())

#lets fill with lower level

C_TEACHER['IN_ESPECIALIZACAO'].fillna(0, inplace = True)
C_TEACHER['IN_MESTRADO'].fillna(0, inplace = True)
C_TEACHER['IN_DOUTORADO'].fillna(0, inplace = True)

number of teacher who work at an school listed at ENEM and do not have study level assigned: 2602384


### New Features

ENEM

In [92]:
#Final Media, Target
ENEM['NU_NOTA_GERAL'] = (ENEM.NU_NOTA_CN + ENEM.NU_NOTA_CH + ENEM.NU_NOTA_LC + ENEM.NU_NOTA_MT + ENEM.NU_NOTA_REDACAO)/5
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

75.00167278973309 % lowers quartis


SCHOOL CENSUS - Itś better include after merge all datasets years.


TEACHER CENSUS

In [93]:
C_TEACHER.isnull().sum()

CO_PROFESSOR         0
CO_ESCOLA            0
IN_ESPECIALIZACAO    0
IN_MESTRADO          0
IN_DOUTORADO         0
dtype: int64

In [94]:
print('number of teacher who work at an school listed at ENEM and do not have study level assigned:',
      C_TEACHER['CO_ESCOLA'].isin(ENEM['CO_ESCOLA']).sum())

#lets fill with lower level

C_TEACHER['IN_ESPECIALIZACAO'].fillna(0, inplace = True)
C_TEACHER['IN_MESTRADO'].fillna(0, inplace = True)
C_TEACHER['IN_DOUTORADO'].fillna(0, inplace = True)

number of teacher who work at an school listed at ENEM and do not have study level assigned: 2602384


In [95]:
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
print('before',C_TEACHER.shape[0])
C_TEACHER.drop_duplicates(subset=['CO_PROFESSOR', 'CO_ESCOLA'], inplace=True)
C_TEACHER.drop('CO_PROFESSOR', axis=1, inplace = True)
print('after',C_TEACHER.shape[0])

before 11471638
after 3176065


In [96]:
C_TEACHER['TITULACAO'] = 0


C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 1

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_DOUTORADO ==1.0), 'TITULACAO'] = 3

SUM= C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform(sum)
COUNT =  C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform('count')
C_TEACHER['TITULACAO'] = (SUM)/(COUNT*3)   

#transformin to school grain
C_TEACHER.drop_duplicates('CO_ESCOLA', inplace = True)

## STUDENT GRAIN 


In [97]:
# teacher to Schoool Grain
print('C_SCHOOL',C_SCHOOL.shape)
CENSO19 = pd.merge(C_SCHOOL, C_TEACHER, on = 'CO_ESCOLA', how = 'inner')
print('C_SCHOOL + C_TEACHER',CENSO19.shape)

#Now, all to Student Grain
print('ENEM',ENEM.shape)
ENEM_CENSO_19 = pd.merge(ENEM, CENSO19, on = 'CO_ESCOLA', how = 'inner')
print('FINAL',ENEM_CENSO_19.shape)

ENEM_CENSO_19.to_feather('2016_STUDENT.feather')

C_SCHOOL (279358, 26)
C_SCHOOL + C_TEACHER (187891, 30)
ENEM (1270333, 17)
FINAL (578654, 46)


In [98]:
ENEM_CENSO_19.dtypes

CO_ANO                           uint16
CO_ESCOLA                         int32
NU_IDADE                          int32
TP_SEXO                           int64
TP_COR_RACA                       int64
TP_ST_CONCLUSAO                   uint8
NU_NOTA_CN                      float32
NU_NOTA_CH                      float32
NU_NOTA_LC                      float32
NU_NOTA_MT                      float32
NU_NOTA_REDACAO                 float32
EDU_PAI                           int64
EDU_MAE                           int64
QT_PESSOAS_CASA                   int32
RENDA_MENSAL                      int64
NU_NOTA_GERAL                   float32
TARGET                            int64
CO_MUNICIPIO                      int64
CO_UF                             int64
CO_DEPENDENCIA_ADM                int64
IN_AGUA_INEXISTENTE             float64
IN_ESGOTO_INEXISTENTE           float64
IN_ENERGIA_INEXISTENTE          float64
IN_LABORATORIO_INFORMATICA      float64
IN_LABORATORIO_CIENCIAS         float64


In [99]:
C_TEACHER.isnull().sum()

CO_ESCOLA            0
IN_ESPECIALIZACAO    0
IN_MESTRADO          0
IN_DOUTORADO         0
TITULACAO            0
dtype: int64

## SCHOOL GRAIN


In [100]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL', 'NU_IDADE', 'NU_NOTA_CN',
             'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','NU_NOTA_GERAL' ]

for i in fields:   
    
            ENEM[i] = ENEM.groupby('CO_ESCOLA')[i].transform('mean')

In [101]:
# SCHOOL GRAIN AND SHOW THE NEW NOTA DISTRIBUTION
ENEM.drop_duplicates('CO_ESCOLA', inplace=True)
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

71.95532941254213 % lowers quartis


In [102]:
#recalculating target for new  Nota Geral distribution 
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

74.99834798123307 % lowers quartis


In [103]:
print('censo before',CENSO19.shape)
print('enem_school before', ENEM.shape)
CENSO_ENEM_19 = pd.merge(CENSO19, ENEM, on='CO_ESCOLA', how='inner')
print(CENSO_ENEM_19.shape)

CENSO_ENEM_19.to_feather('2015_SCHOOL.feather')

censo before (187891, 30)
enem_school before (30266, 17)
(13391, 46)


In [104]:
CENSO_ENEM_19.isnull().sum()


CO_ESCOLA                       0
CO_MUNICIPIO                    0
CO_UF                           0
CO_DEPENDENCIA_ADM              0
IN_AGUA_INEXISTENTE             0
IN_ESGOTO_INEXISTENTE           0
IN_ENERGIA_INEXISTENTE          0
IN_LABORATORIO_INFORMATICA      0
IN_LABORATORIO_CIENCIAS         0
IN_SALA_ATENDIMENTO_ESPECIAL    0
IN_BIBLIOTECA                   0
IN_SALA_LEITURA                 0
IN_BANHEIRO_FORA_PREDIO         0
IN_BANHEIRO_PNE                 0
IN_DEPENDENCIAS_PNE             0
QT_SALAS_UTILIZADAS             0
QT_EQUIP_TV                     0
QT_EQUIP_DVD                    0
QT_EQUIP_COPIADORA              0
QT_EQUIP_IMPRESSORA             0
QT_COM_ADMINISTRATIVO           0
QT_COMP_ALUNO                   0
IN_INTERNET                     0
IN_BANDA_LARGA                  0
QT_FUNCIONARIOS                 0
IN_ALIMENTACAO                  0
IN_ESPECIALIZACAO               0
IN_MESTRADO                     0
IN_DOUTORADO                    0
TITULACAO     